In [168]:
import requests
from bs4 import BeautifulSoup
import json
import translators as ts
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
import sys
import xlrd
import cv2
import pandas as pd
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from datetime import date
import numpy as np

meta = ""
fonts = ""
root = ""

with open('D:\\settings.txt') as f:
    lines = f.readlines()
    for line in lines:        
        if (line.find("meta")!=-1):            
            meta = line[6:].strip()
        if (line.find("root")!=-1):            
            root = line[6:].strip()
        if (line.find("fonts")!=-1):
            #print(line[7:])
            fonts = line[7:].strip()

meta_file = meta
df = pd.read_excel(meta_file, sheet_name = "Master")
    
    

In [148]:
def print_headlines(response_text):
    headlines_ = []
    headlines_hindi = []
    soup = BeautifulSoup(response_text, 'lxml')
    headlines = soup.find_all(attrs={"itemprop": "headline"})
    for headline in headlines:
        headlines_.append(headline.text)
        headlines_hindi.append(ts.google(headline.text, from_language='en', to_language='hi'))
    return headlines_,headlines_hindi

url = 'https://inshorts.com/en/read'
response = requests.get(url)
newsList,newsListHindi  = print_headlines(response.text)

In [149]:
def get_FileName(input):   
    s_index = input.find(".png")    
    return input[s_index-5:s_index+4]

def find_end(image):
    w = image.shape[1]
    final = 0
    try:
        for y in range (w-25,0,-1):
            roi = image[0:160,y:y+25]
            sum = 0
            for i in range(160):
                for j in range(25):
                    #print(roi[i,j])
                    k = roi[i,j]
                    if k[0]==255 and k[1]==255 and k[2]==255:
                        flag =0
                    else:
                        flag =1
                    sum = sum+flag
            #rint(y,sum)
            if sum==0:
                final = y
                return final+15
    except:
        a=0
    return final+15

def make_tankrifont(input_text):
    im = cv2.imread(fonts + "F_S.png")
    im_b = cv2.imread(fonts + "F_67.png")


    len_ = len(input_text)
    skip_next = 0

    for index in range (len_):

        im_t = cv2.imread(fonts + "F_67.png")
        im_b = cv2.imread(fonts + "F_67.png")
        prefix = ""
        marker = ""
        if skip_next!=0:
            skip_next=skip_next-1
            continue

        now_ = input_text[index]
        if index+1<len_:
            next_ = ord(input_text[index+1])
            #print("Next is " + str(next_))
            if next_ in (2366,2375,2376,2379,2380,2370,2306):
                #print("Yes")
                marker = "T"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    prefix = "A"
            if next_ in (2369,2370,2381):
                marker = "B"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    marker = "BT"
            if next_ == 2367:
                marker = "LT"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    prefix = "A"
            if next_ == 2368:
                marker = "RT"
                skip_next = 1
                if (index+2<len_) and (ord(input_text[index+2]) in (2305,2306)):
                    skip_next = 2
                    prefix = "A"


        uni_ = ord(now_)       
        x = df.loc[df['Unicode_D'] == uni_]
        filename = (get_FileName(str(x['File_Name']))).strip()
        if (filename.find("png"))==-1:
            print(now_,uni_,filename)            
            filename = "F_99.png"

        img = cv2.imread(fonts + filename)

        if (marker != ""):
            xn = df.loc[df['Unicode_D'] == next_]
            filename = (get_FileName(str(xn['File_Name']))).strip()
            #print(now_,uni_,filename)
            #print('D:\\Project Himachal\\Handwritten Sample\\Fonts_Resized\\' + filename)
            if (marker=="T"):
                im_t = cv2.imread(fonts + prefix+filename)
                #print("Hari Om")
            if (marker=="B"):
                im_b = cv2.imread(fonts + filename)

            if (marker=="LT"):
                im_t = cv2.imread(fonts + prefix + 'F_14T.png')
                im_temp = cv2.imread(fonts + 'F_14L.png')
                img = cv2.hconcat([im_temp,img])           

            if (marker=="RT"):
                im_t = cv2.imread(fonts + prefix + 'F_14T.png')
                im_temp = cv2.imread(fonts + 'F_14L.png')
                img = cv2.hconcat([img,im_temp]) 

            if (marker=="BT"):
                im_t = cv2.imread(fonts + 'F_30.png')
                im_b = cv2.imread(fonts + filename)            
        
        if (uni_==2371):
            im_t = cv2.imread(fonts + 'F_14T.png')

        w = img.shape[1]
        top = cv2.resize(im_t, (w, 30))
        bottom = cv2.resize(im_b, (w, 30))
        #print(img.shape)
        if (uni_) not in (2310,2311,2312,2313,2314,2320,2323,2324):
            img = cv2.vconcat([top, img])
        #print(img.shape)
        img = cv2.vconcat([img,bottom])
        #print(img.shape)
        #v2.imshow('temp', img)
        im = cv2.hconcat([im, img])
    return im

def tile_image(image,tile_width):
    
    #tile_width = 5000
    #tile_width = tile_width - 50
    im_b = cv2.imread(fonts + 'F_67.png')
    im_le = cv2.imread(fonts+ 'F_LineEnd.png')
    w = image.shape[1]
    
    img = cv2.resize(im_b, (tile_width, 30))
    
    
    y = 0
    
    while y<w:    
        #print(y)
        if(y+tile_width<w):
            im = image[0:160,y :y + tile_width]
            far_end = find_end(im)
            #print(far_end)
            im = image[0:160,y:y + far_end]
            image_pad = cv2.resize(im_b, (tile_width-far_end, 160))
            #print(im.shape)            
            #print(image_pad.shape)  
            im = cv2.hconcat([im, image_pad])
            pre_ = tile_width-far_end
            #print(im.shape) 
            y = y + far_end
        else:
            im = image[0:160,y:w]
            #print(im.shape)
            #print(w-y)
            image_pad = cv2.resize(im_b, (tile_width-(w-y), 160))
            im = cv2.hconcat([im, image_pad])
            #print(im.shape)
            y=y+tile_width
        img = cv2.vconcat([img,im])
    
    return img

In [194]:
im_b = cv2.imread('D:\\Project Himachal\\Data\\Kangri\\News\\blank.png')

img = Image.open('D:\\Project Himachal\\Data\\Kangri\\News\\Title.png')
I1 = ImageDraw.Draw(img)
myFont = ImageFont.truetype('D:\\Project Himachal\\Data\\Kangri\\News\\StonyIslandNF-Bold.ttf', 100) 
myFont2 = ImageFont.truetype('D:\\Project Himachal\\Data\\Kangri\\News\\StonyIslandNF-Bold.ttf', 50) 
#myFont_hi = ImageFont.truetype('D:\\Project Himachal\\Data\\Kangri\\News\\krdv011.ttf', 100) 
today = date.today()
d2 = today.strftime("%b %d, %Y")
#d2_hi = ts.google(d2, from_language='en', to_language='hi')
#print(d2_hi)
I1.text((150, 250), d2, font=myFont,fill =(0, 0, 0))
I1.text((1950, 280), "Beta version, (c) Prawaal, Navneet Goyal & Poonam Goyal", font=myFont2,fill =(180, 180, 180))
img.save("D:\\Project Himachal\\Data\\Kangri\\News\\TitleToday.png")

In [196]:

im_top = cv2.imread('D:\\Project Himachal\\Data\\Kangri\\News\\TitleToday.png')
im_vspace = tile_image(make_tankrifont(" "),3015) 
in_v = im_vspace
im1_flag = 0
im2_flag = 0
full_paper =  im_top

for index,news in enumerate(newsListHindi):
    #print(news)
    im = tile_image(make_tankrifont(news),1500) 
    hieght = im.shape[0]
    width = im.shape[1] 
    #print(hieght)
    if ((hieght<670) and (hieght>500)):
        im = cv2.vconcat([im,im_b]) 
    
    if ((hieght<500) and (hieght>340)):
        im = cv2.vconcat([im,im_b]) 
        im = cv2.vconcat([im,im_b]) 
        
    outputImage = cv2.copyMakeBorder(
                 im, 
                 20, 
                 20, 
                 20, 
                 20, 
                 cv2.BORDER_CONSTANT, 
                 value=(150,150,150))
    #cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\Ind_' + str(index) +'.png', outputImage  )
    if (im1_flag==0) and (im2_flag==0):
        #print("(im1_flag==0) and (im2_flag==0)")
        #im1 = outputImage
        cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\temp_left.png', outputImage  )
        im1_flag = 1
        continue
    if (im1_flag==1) and (im2_flag==0):
        #print("(im1_flag==1) and (im2_flag==0)")
        cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\temp_right.png', outputImage  )
        #im2 = outputImage
        im2_flag =1
        continue
    if (im1_flag==1) and (im2_flag==1):
        #print("(im1_flag==1) and (im2_flag==1)")
        im_left = cv2.imread('D:\\Project Himachal\\Data\\Kangri\\News\\temp_left.png')
        im_right = cv2.imread('D:\\Project Himachal\\Data\\Kangri\\News\\temp_right.png')
        im_h = cv2.hconcat([im_left,im_right]) 
        outputImage = cv2.copyMakeBorder(
                 im_h, 
                 0, 
                 0, 
                 20, 
                 20, 
                 cv2.BORDER_CONSTANT, 
                 value=(150,150,150))
        #cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\News_' + str(index) +'.png', outputImage  )
        try:
            full_paper = cv2.vconcat([full_paper,outputImage]) 
        except:
            full_paper = im_top
        #print("Image Saved")
        im1_flag = 0
        im2_flag = 0
        #in_v = cv2.vconcat([im_v,im_h]) 
        
        if(index%14==0) and (index!=0):
            cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\News_' + str(index) +'.png', full_paper  )   
            full_paper = im_top

hieght = full_paper.shape[0]
img_buffer = np.zeros((3959-hieght,3120,3), dtype=np.uint8)
img_buffer.fill(255)
cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\Buffer.png', img_buffer  )   
full_paper = cv2.vconcat([full_paper,img_buffer])             
cv2.imwrite('D:\\Project Himachal\\Data\\Kangri\\News\\News_' + str(index) +'.png', full_paper  )   

़ 2364 
% 37 
़ 2364 
ऋ 2315 
ष 2359 
: 58 
- 45 
़ 2364 
$ 36 
: 58 
ॉ 2377 
़ 2364 
: 58 
़ 2364 
$ 36 
' 39 
' 39 
' 39 
' 39 
: 58 
ॉ 2377 
ॉ 2377 
ॉ 2377 
₹ 8377 
- 45 
' 39 
' 39 
' 39 
़ 2364 
% 37 
₹ 8377 
% 37 
़ 2364 


True

In [153]:
newsListHindi

['गुजरात के वडोदरा में आरओ प्लांट में बड़े पैमाने पर आग टूट जाती है, कई दुकानें आंतीं',
 'Badshah मसाला में 51% हिस्सेदारी खरीदने के लिए Dabur। 587.5 करोड़',
 'यूके पीएम ऋषि सुनाक भारतीय राजनीति में एक बैकबेंचर थे: थारूर',
 '27-28 अक्टूबर को पानी की आपूर्ति के व्यवधान का सामना करने के लिए दिल्ली के कई हिस्सों',
 'पार्किंग के मुद्दे पर आदमी को पीटा गया, सिर ईंट के साथ धराशायी हो गया',
 'उबेर ने मुंबई महिला को ober 20,000 का भुगतान करने के लिए कहा, जो उड़ान से चूक गई थी',
 'थाई बिजनेसवूमन मिस यूनिवर्स संगठन $ 20 मिलियन के लिए खरीदता है',
 'Unapologetic वीरता: RRR की वैश्विक सफलता के पीछे के कारण राजमौली',
 'स्नूप डॉग प्रति दिन लगभग 75 से 150 जोड़ों को धूम्रपान करता है: रैपर के स्टाफ सदस्य',
 'करमन कौर थांडी भारत का नहीं बन जाता है। 1 महिला एकल टेनिस खिलाड़ी',
 'Apple अमेरिका में iPhone उपयोगकर्ताओं के लिए स्वच्छ ऊर्जा चार्जिंग को रोल करना शुरू करता है',
 'मेटा प्लेटफार्मों ने तुर्की प्रतियोगिता प्राधिकरण द्वारा $ 18.6 मिलियन का जुर्माना लगाया',
 "मेटा विज्ञापन नीति के साथ 'दूसरों को कम क